<a href="https://colab.research.google.com/github/algarcser/TOO-proyecto/blob/main/entrega.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Este cuaderno contrendrá el projecto final de AAII20223 realizar por Alejandro Guarnizo de la Torre y Álvaro Gómez Troitño. 

# Criterios este trabajo
Requisitos generales:
  Grupos de 1 a 3 personas (ideal 3).
  Probar con al menos 3 modelos distintos (si el grupo es de 2 personas el requisito es de 2 modelos).
  Cada una de las partes ha de estar bien documentada y descrita.
  Requisitos mínimos del trabajo (para aprobar el trabajo final):
  Selección de un dataset (ver sugerencias a continuación)

Breve preprocesamiento de datos: no es el objetivo de esta asignatura pero sí que es necesario realizar un breve estudio de conocer cómo son los datos (como los vistos en las prácticas). En caso de columnas incompletas, con valores nulos, o datos con dimensión muy alta, se permite eliminar estas columnas y no tener en cuenta algunas de las características.
Para cada modelo:
  - Datos entrada: estudio del balanceo de los datos y aplicar si es el caso una técnica de balanceo. (Si los datos están balanceados, reducir de forma artificial una de las clases y aplicar una técnica para balancear. En este caso, continuar con el conjunto original el resto de ejercicios, es decir con el conjunto balanceado inicial.)
  - Incluir técnicas de validación (aplicar al menos 1 técnica).
  - Estudio de hiperparámetros (aplicar al menos 1 técnica) y creación del modelo
  - Evaluación del modelo con más de una métrica de las vistas en clase
  - Comparación de modelos
  - Hacer un ensemble con los modelos elegidos. Comparar los resultados con hacerlos de manera independiente.
  
**Ejercicios para alcanzar mayor nota**:
  
  - Utilizar más de una técnica de balanceo
  - Ampliar las técnicas del estudio de hiperparámetros
  - Ampliar el número de modelos
  - Utilizar varios métodos de ensemble
  - Aplicar más de una técnica de interpretabilidad.

# Carga de datos

A continuación vamos a cargar el dataset con el cual vamos a realizar este trabajo. 
En este trabajo se ha escogido el dataset proporcionado por Kaggle en el siguiente enlace: https://www.kaggle.com/denisadutca/customer-behaviour

Este dataset tiene como principal objetivo el establecer si un posible comprador realizará una compra en función de una seria de características que conocemos de la persona. 

Conocemos los siguientes campos de una persona: 
- Id
- Género
- Edad
- Salario estimado
- Comprado? (está será nuestra variable objetivo)

Con esta información tenemos que intertan predecir sin un cliente comprará o no en la tienda. 

Vamos a cargar los datos y hacer una previsualización de los datos

Cargamos algunas librerías que vamos a usar

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
# Leer dataset
url = "/content/Customer_Behaviour.csv"
df = pd.read_csv(url, sep=",")
 
# Mostrar las n primeras filas del dataset
df.head(5)

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0


Según la información proporcionada por Kaggle, este dataset no contiene ninguna fila con datos vacios y tampoco tienen filas repetidas o no validas, por lo tanto no vamos a preocuparnos con estos aspectos en este trabajo y que que no hace falta

Hemos cargado correctamente el dataset
Vamos a realizar una pequeña labor de preprocesamiento.

Como hemos comentado nuestra variable objetivo será la columnas Purchased, que indica que si un cliente o no, realizó una compra.

Vamos a ver la distribución de nuestro dataset y si está balanceado o no

In [5]:
df['Purchased'].value_counts()

0    257
1    143
Name: Purchased, dtype: int64

Como podemos ver, no está totalmente balanceado nuestro dataset, es decir que deberemos de aplicar algunas técnicas de balanceo para ver si podemos mejorar la eficiencia de nuestros algoritmos

si vemos cuantos que porcentaje de datos corresponde a cada clase

In [6]:
df['Purchased'].value_counts() / len(df) * 100

0    64.25
1    35.75
Name: Purchased, dtype: float64

Observamos que tenemos aproximadamente 1/3 de nuestros datos corresponden a compras realizadas, y 2 / de nuestros datos corresponden a clientes que no han comprado nada. 

Antes de continuar vamos a conventir una de nuestras variables, el género de variable categórica, a variable numérica, para poder trabajar más facilmente con el dataset

In [7]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()

df['Gender'] = le.fit_transform(df['Gender'])

df.head()

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,1,19,19000,0
1,15810944,1,35,20000,0
2,15668575,0,26,43000,0
3,15603246,0,27,57000,0
4,15804002,1,19,76000,0


Hemos convertirdo nuestra variable género a una variable numérica. 
Se le ha asignado 1 a hombre y 0 a mujer tal y como podemos ver si compararmos las filas mostradas anteriormente y ahora

Ahora bien, la parte importante que hemos observado anteriormente nuestro dataset está desbalanceado y por tanto vamos a aplicar varias técnicas para que llegemos a una proporción más cercana a 50/50

Vamos a emplear dos técnicas de forma conjunta para reducir la clase mayoritaria y aumentar la clase minoritaria, e forma que teneremos un dataset mejor balanceado. 

Si es necesario emplearemos otras técnicas de balanceo para poder comprarar resultados mejor
Vamos a escoger un punto media entre ambas clases para Como tenemos una diferencia de 100 filas en total. 
Escogamos por ejemplo 200 elementos para cada clase. 

In [8]:
from sklearn.utils import resample

In [9]:
# Separar las clases mayoritaria y minoritaria
df_majority = df[df.Purchased==0]
df_minority = df[df.Purchased==1]

print(f"tenemos {len(df_majority)} en el mayor y {len(df_minority)} en el menor")


# Aumentar la muestra de la clase minoritaria
df_minority_upsampled = resample(df_minority, 
                                 replace=True,     # muestra con remplazamiento
                                 n_samples=200,    # número de muestras de la clase mayoritaria
                                 random_state=42) # semilla para que los resultados sean reproducibles

# Disminuir la clase mayoritaria
df_majority_downsampled = resample(df_majority, 
                                 replace=False,    # muestra sin remplazamiento
                                 n_samples=200,     # número de muestras de la clase minoritaria
                                 random_state=42) # semilla para que los datos sean reproducibles

# combinamos los dos dataframes generados 
df_mixto = pd.concat([df_minority_upsampled, df_majority_downsampled]) 

# veamos la proporción que hemos obtenido
print("La nueva proporción es:")
df_mixto['Purchased'].value_counts() / len(df_mixto) * 100

tenemos 257 en el mayor y 143 en el menor
La nueva proporción es:


1    50.0
0    50.0
Name: Purchased, dtype: float64

Otra de las opciones serían generar datos sinéticos empleado Smote o ADASYN para la clase minoritaria, mientras que disminumos la clase mayoritaria, de forma que evitemos tener que generar muchos datos sintéticos. 

Resaltar que podemos hacer alguna labor adicicional de preprocesamiento de los datos, por ejemplo es muy comun standarizar los datos ya que hay algunos modelos que funcionan mejor cuando los datos son número de una escala más pequeña. 
Si vemos que es necesario realizaremos este labor de estandarizar los datos más adelante. 

# Estudio modelos

Una vez ya tenemos nuestro dataset generado y listo para ser estudiamo, vamos a continuación a realizar diferentes modelos y compararlos entre ellos para poder obtener conclusiones y el mejor modelo funcional que podamos generar. 

Vamos a usar un gridSearch para buscar los mejores parámetros para el parámetro elegido

Primeramente vamos a generar nuestros conjuntos de entrenamiento y validación como es costumbre. para poder comparar los diferentes algoritmos. Usaremos el split test para dividir los datos en 2 conjuntos uno de test y otro de 

In [ ]:
%%time
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV


param_grid = {
    # 12 (3×4) combinaciones de hiperparámetros
    'n_estimators': [30, 100, 300],
    'max_features': [3, 6, 8, 'auto']
    }   

forest_reg = RandomForestRegressor(random_state=42) 
# entrena con 3 folds, es decir un total de 12*3=36 rondas de entrenamiento 
grid_search_RF = GridSearchCV(forest_reg, param_grid, cv=3,
                           scoring='neg_mean_squared_error',
                           refit= True, verbose=2, n_jobs=-1)
grid_search_RF.fit(X_train, y_train)


cvres = grid_search_RF.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(f"El RMSE para el modelo con los parámetros {params} ha sido {np.sqrt(-mean_score)}")

    